In [1]:
import os
import math
import numpy as np
import scipy.signal as sig
import matplotlib.pyplot as plt
import matplotlib.colors as colors
import matplotlib.ticker as ticker
from matplotlib import cm
from scipy import stats
from scipy import optimize
import numpy.polynomial.polynomial as poly
from bisect import bisect_left

In [2]:
def import_file(path, limit_low=None, limit_high=None):

    spectrum = np.genfromtxt(path, delimiter=",")
    spectrum = np.transpose(spectrum)
    wavenumbers = spectrum[0]
    intensities = spectrum[1]

    if limit_low is not None:
        limit_low_index = list(wavenumbers).index(limit_low)
    else:
        limit_low_index = 0
        limit_low = wavenumbers[0]

    if limit_high is not None:
        limit_high_index = list(wavenumbers).index(limit_high)
    else:
        limit_high_index = len(wavenumbers)
        limit_high = wavenumbers[-1]

    wavenumbers = wavenumbers[limit_low_index:limit_high_index]
    intensities = intensities[limit_low_index:limit_high_index]
    return wavenumbers, intensities

def import_directory(path, limit_low=None, limit_high=None):
    # files = os.listdir(path)

    # for filename in files:
    #     np.genfromtxt(filename, delimiter=",")
    pass

In [11]:
wavenumbers, intensities = import_file("spectra/E (1).TXT", 450, 1600)

In [12]:
intensities_sg0 = sig.savgol_filter(intensities, 
                                    window_length=13, 
                                    polyorder=3, 
                                    deriv=0)

intensities_sg1 = sig.savgol_filter(intensities, 
                                    window_length=13, 
                                    polyorder=3, 
                                    deriv=1)

intensities_sg2 = sig.savgol_filter(intensities, 
                                    window_length=13, 
                                    polyorder=3, 
                                    deriv=2)

intensities_sg3 = sig.savgol_filter(intensities, 
                                    window_length=13, 
                                    polyorder=3, 
                                    deriv=3)

In [13]:
zeros_sg3 = np.diff(np.sign(intensities_sg3))
peaks = np.where(zeros_sg3 > 0)[0]
peaks = peaks[intensities_sg2[peaks] / intensities_sg0[peaks] < 0]

peak_condensing = []
peaks_new = []
for i in range(len(wavenumbers)-1):
    if i in peaks:
        peak_condensing.append(i)
    if intensities_sg2[i] > 0 and len(peak_condensing) > 0:
        peaks_new.append(int(np.mean(peak_condensing)))
        peak_condensing = []
if len(peak_condensing) > 0:
    peaks_new.append(int(np.mean(peak_condensing)))
peaks_new = np.asarray(peaks_new)            

In [14]:
## Basislinie Versuch 3 

# Liste an Minima erstellen
zeros_sg1 = np.diff(np.sign(intensities_sg1))
mins = list(np.where(zeros_sg1 > 0)[0])
mins.insert(0, 0)
mins.append(len(wavenumbers)-1)

intervals = {}
for i in range(len(mins)-1):
    interval = (mins[i], mins[i+1]) # Immer 2 benachbarte Minima sind ein Intervall
    
    #Ein Minimum davor und danach hinzufügen -> für Polynom
    if i == 0:
        baseline_points = [0] + mins[:3]
        baseline_wns = wavenumbers[baseline_points]
        baseline_wns[0] = 2*baseline_wns[1] - baseline_wns[2]
    elif i == len(mins)-2:
        baseline_points = mins[-3:] + [mins[-1]]
        baseline_wns = wavenumbers[baseline_points]
        baseline_wns[-1] = 2*baseline_wns[-2] - baseline_wns[-3]
    else:
        baseline_points = mins[i-1:i+3]
        baseline_wns = wavenumbers[baseline_points]
    baseline_ints = intensities_sg0[baseline_points]
    
    # Polynom fitten und Basislinie berechnen
    interval_relative = [i-baseline_points[0] for i in interval]
    fit = poly.polyfit(baseline_wns, baseline_ints, 3)
    x = wavenumbers[baseline_points[0]:baseline_points[-1]]
    local_baseline = poly.polyval(x, fit)
    
    wns = wavenumbers[interval[0]:interval[1]]
    ints = intensities_sg0[interval[0]:interval[1]]
    ints_corrected = ints - local_baseline[interval_relative[0]:interval_relative[1]]
    
    # Negative Bereiche korrigieren
    mins_new = sig.argrelmin(ints_corrected)[0]
    
    mins_new = [0] + [x for x in mins_new if ints_corrected[x] < 0] + [len(ints_corrected)-1]
    
    baseline2_wns = wns[mins_new]
    baseline2_ints = ints_corrected[mins_new]
    fit2 = poly.polyfit(baseline2_wns, baseline2_ints, len(mins_new)-1)
    local_baseline2 = poly.polyval(x, fit2)
    ints_corrected = ints_corrected - local_baseline2[interval_relative[0]:interval_relative[1]]
    local_baseline += local_baseline2
    
    intervals[interval] = {"wavenumbers":wns,
                           "intensities":ints,
                           "baseline_wns":x,
                           "baseline_ints":local_baseline,
                           "corrected_ints":ints_corrected,
                           "rel_int":interval_relative}


In [25]:
## Peakfinding

for i, d in intervals.items():
    print(i)
    
    try:
        intensities_sg2 = sig.savgol_filter(d["corrected_ints"], 
                                            window_length=13, 
                                            polyorder=3, 
                                            deriv=2)
    except ValueError:
        intensities_sg2 = np.zeros(len(d["corrected_ints"]))

    plt.plot(d["wavenumbers"], intensities_sg2)
    peaks = sig.argrelmin(intensities_sg2, order=10)[0]
    peaks = [peak for peak in peaks if intensities_sg2[peak] < 0]
    
    peak_condensing = []
    peaks_new = []
    for i in range(len(intensities_sg2)):
        if i in peaks:
            peak_condensing.append(i)
        if intensities_sg2[i] > 0 and len(peak_condensing) > 0:
            peaks_new.append(int(np.mean(peak_condensing)))
            peak_condensing = []
    if len(peak_condensing) > 0:
        peaks_new.append(int(np.mean(peak_condensing)))
    peaks = peaks_new
    
    valleys = [0]
    
    for i in range(len(peaks)-1):
        valleys.append(peaks[i] + np.argmax(intensities_sg2[peaks[i]:peaks[i+1]]))
    
    valleys.append(len(intensities_sg2)-1)
    
    peak_intervals = []
    for i in range(len(peaks)):
        peak_intervals.append((valleys[i], valleys[i+1]))
        
    peak_heights = d["corrected_ints"][peaks]
    
    peaks = [peak for _,peak in sorted(zip(peak_heights,peaks))]
    peak_intervals = [interval for _,interval in sorted(zip(peak_heights,peak_intervals))]
    peaks.reverse()
    peak_intervals.reverse()
    peak_wns = d["wavenumbers"][peaks]
    peak_heights = d["corrected_ints"][peaks]
    
    d["peaks"] = peaks
    d["peak_wns"] = peak_wns
    d["peak_heights"] = peak_heights
    d["peak_intervals"] = peak_intervals

(0, 9)
(9, 69)
(69, 197)
(197, 293)
(293, 495)
(495, 647)
(647, 682)
(682, 705)
(705, 785)
(785, 890)
(890, 960)
(960, 1091)
(1091, 1152)
(1152, 1248)
(1248, 1428)
(1428, 1633)
(1633, 1666)
(1666, 1918)
(1918, 1961)
(1961, 2089)
(2089, 2108)
(2108, 2299)


In [26]:
%matplotlib
for i, d in intervals.items():
    print(i)
    fig, ax = plt.subplots(2,1)
    ax[0].plot(d["wavenumbers"], d["intensities"], linewidth=1, color="blue")
    ax[0].plot(d["baseline_wns"], d["baseline_ints"], linewidth=1, color="red")
    ax[0].fill_between(d["wavenumbers"],
                       d["intensities"],
                       d["baseline_ints"][d["rel_int"][0]:d["rel_int"][1]])
    ax[1].plot(d["wavenumbers"], d["corrected_ints"], linewidth=1, color="blue")
    ax[1].hlines(0, d["wavenumbers"][0], d["wavenumbers"][-1])
    ax[1].fill_between(d["wavenumbers"],
                       d["corrected_ints"],
                       0)
    if len(d["peak_heights"]) > 0:
        ax[1].vlines(d["peak_wns"], 0, d["peak_heights"][0]*1.1, color="red")
    
    #ax.vlines(values["wavenumbers"][values["peaks"]], np.min(values["intensities"])*0.9, np.max(values["intensities"])*1.1, color="red")
    #ax.vlines(values["wavenumbers"][values["valleys"]], np.min(values["intensities"])*0.9, np.max(values["intensities"])*1.1, color="blue")

Using matplotlib backend: Qt5Agg
(0, 9)
(9, 69)
(69, 197)
(197, 293)
(293, 495)
(495, 647)
(647, 682)
(682, 705)
(705, 785)
(785, 890)
(890, 960)
(960, 1091)
(1091, 1152)
(1152, 1248)
(1248, 1428)
(1428, 1633)
(1633, 1666)
(1666, 1918)
(1918, 1961)
(1961, 2089)
(2089, 2108)
(2108, 2299)


<ipython-input-26-e4d61211cadd>:4: RuntimeWarning: More than 20 figures have been opened. Figures created through the pyplot interface (`matplotlib.pyplot.figure`) are retained until explicitly closed and may consume too much memory. (To control this warning, see the rcParam `figure.max_open_warning`).
  fig, ax = plt.subplots(2,1)


In [27]:
def gaussian(x, amplitude, mean, stddev):
    return (amplitude * np.exp(-((x - mean) / 4 / stddev)**2))

for i, d in intervals.items():
    print(i)
    corrected_ints = d["corrected_ints"][:]
    gauss_all = []
    for j in range(len(d["peaks"])):
        peak = d["peaks"][j]
        peak_interval = d["peak_intervals"][j]
        p0 = [corrected_ints[peak], d["wavenumbers"][peak], d["wavenumbers"][-1] - d["wavenumbers"][0]]
        p = optimize.curve_fit(gaussian,
                               d["wavenumbers"][peak_interval[0]:peak_interval[1]],
                               corrected_ints[peak_interval[0]:peak_interval[1]],
                               p0=p0)[0]
        
        gauss = []
        for wn in d["wavenumbers"]:
            gauss.append(gaussian(wn, p[0], p[1], p[2]))
        gauss = np.array(gauss)
        corrected_ints = corrected_ints - gauss
        if j > 0:
            gauss += gauss_all[j-1]
        gauss_all.append(gauss)
        
    d["gauss"] = gauss_all

(0, 9)
(9, 69)
(69, 197)
(197, 293)
(293, 495)
(495, 647)
(647, 682)
(682, 705)
(705, 785)
(785, 890)
(890, 960)
(960, 1091)
(1091, 1152)
(1152, 1248)
(1248, 1428)
(1428, 1633)
(1633, 1666)
(1666, 1918)
(1918, 1961)
(1961, 2089)
(2089, 2108)
(2108, 2299)


In [28]:
%matplotlib
for i, d in intervals.items():
    fig, ax = plt.subplots(2,1)
    ax[0].plot(d["wavenumbers"], d["intensities"], linewidth=1, color="blue")
    ax[0].plot(d["baseline_wns"], d["baseline_ints"], linewidth=1, color="red")
    ax[0].fill_between(d["wavenumbers"],
                       d["intensities"],
                       d["baseline_ints"][d["rel_int"][0]:d["rel_int"][1]])
    ax[1].plot(d["wavenumbers"], d["corrected_ints"], linewidth=1, color="blue")
    for gauss in d["gauss"]:
        ax[1].plot(d["wavenumbers"], gauss, linewidth=1, color="black")
    ax[1].hlines(0, d["wavenumbers"][0], d["wavenumbers"][-1])
    ax[1].fill_between(d["wavenumbers"],
                       d["corrected_ints"],
                       0)
    if len(d["peak_heights"]) > 0:
        ax[1].vlines(d["peak_wns"], 0, d["peak_heights"][0]*1.1, color="red")
    
    #ax.vlines(values["wavenumbers"][values["peaks"]], np.min(values["intensities"])*0.9, np.max(values["intensities"])*1.1, color="red")
    #ax.vlines(values["wavenumbers"][values["valleys"]], np.min(values["intensities"])*0.9, np.max(values["intensities"])*1.1, color="blue")

Using matplotlib backend: Qt5Agg


<ipython-input-28-e8ecec737efc>:3: RuntimeWarning: More than 20 figures have been opened. Figures created through the pyplot interface (`matplotlib.pyplot.figure`) are retained until explicitly closed and may consume too much memory. (To control this warning, see the rcParam `figure.max_open_warning`).
  fig, ax = plt.subplots(2,1)
